In [71]:
import re
import praw
import pandas as pd
from urllib.parse import urlparse
from datetime import datetime, timedelta

Create authorized PRAW instance

In [72]:
# Create a Reddit instance
reddit = praw.Reddit(
    client_id='XQe5nwF9-C44Jb2K4KuYVA',
    client_secret='OEukMAkDtaCXJ8D7YQ_OBohd52RXtQ',
    user_agent='Data_Analysis_project'
)

# Access the "buildapcsales" subreddit
subreddit = reddit.subreddit('buildapcsales')

get 1000 posts from top this year (can only retrieve 1000 posts due to reddit limit)

In [73]:
# Get the top posts of this year
top_posts = subreddit.top(time_filter='year', limit=None)

# Create a list to store the post attributes
post_attributes = []

# Retrieve and store the attributes of each post
for post in top_posts:
    post_attributes.append([post.title, post.link_flair_text, post.score, post.created_utc, post.url, str(post.author), post.num_comments])

# Create a pandas DataFrame from the post attributes
df = pd.DataFrame(post_attributes, columns=['Title', 'Flair', 'Score', 'Created UTC', 'URL', 'Author', 'Num_Comments'])

print(df.head(5))

                                               Title                 Flair  \
0  [META] Micro Center is expanding with three ne...                 Meta    
1          [META] EVGA Terminates NVIDIA Partnership                 Meta    
2  [Console] Steam Deck - 64GB/$359 256GB/$476 51...               Console   
3  [META] NVIDIA GeForce RTX 4090 24GB GDDR6X to ...                 Meta    
4  [Monitor] Dell 27” S2721DGF 1440p 165Hz IPS 1m...  Expired :table_flip:   

   Score   Created UTC                                                URL  \
0   3732  1.678456e+09  https://www.pcmag.com/news/electronics-retaile...   
1   2960  1.663356e+09        https://www.youtube.com/watch?v=cV9QES-FUAM   
2   2067  1.678986e+09           https://store.steampowered.com/steamdeck   
3   2023  1.663688e+09  https://www.nvidia.com/en-us/geforce/graphics-...   
4   2005  1.666010e+09  https://www.dell.com/en-us/shop/dell-27-gaming...   

          Author  Num_Comments  
0     ctskifreak           744  
1 

fix date format

In [74]:
# Convert "Created UTC" to datetime object
df['Created Date'] = pd.to_datetime(df['Created UTC'], unit='s').dt.date

# Change "flair" column values to uppercase
df['Flair'] = df['Flair'].str.upper()

# Change "Title" column values to uppercase
df['Title'] = df['Title'].str.upper()

print(df)

                                                 Title                 Flair  \
0    [META] MICRO CENTER IS EXPANDING WITH THREE NE...                 META    
1            [META] EVGA TERMINATES NVIDIA PARTNERSHIP                 META    
2    [CONSOLE] STEAM DECK - 64GB/$359 256GB/$476 51...               CONSOLE   
3    [META] NVIDIA GEFORCE RTX 4090 24GB GDDR6X TO ...                 META    
4    [MONITOR] DELL 27” S2721DGF 1440P 165HZ IPS 1M...  EXPIRED :TABLE_FLIP:   
..                                                 ...                   ...   
995  [GPU] ASUS TUF GAMING AMD RADEON™ RX 6800 XT O...  EXPIRED :TABLE_FLIP:   
996  [GPU] ASUS DUAL RADEON RX 6700 XT 12GB OC EDIT...                   GPU   
997  [GPU] GIGABYTE GAMING GEFORCE RTX 3090 TI 24GB...                   GPU   
998             [GPU] ASUS STRIX RTX 3080 12 GB - $800  EXPIRED :TABLE_FLIP:   
999  [LAPTOP] GIGABYTE - AERO 16" 4K AMOLED GAMING ...                LAPTOP   

     Score   Created UTC               

get date range of dataset

In [75]:
# Find the earliest and latest dates
earliest_date = min(df['Created Date'])
latest_date = max(df['Created Date'])

print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

Earliest Date: 2022-06-09
Latest Date: 2023-06-08


Get website names

In [76]:
# Extract website name from URL
df['Website'] = df['URL'].apply(lambda x: x.split('//')[-1].split('www.')[-1].split('.com')[0])

print(df.head(5))

                                               Title                 Flair  \
0  [META] MICRO CENTER IS EXPANDING WITH THREE NE...                 META    
1          [META] EVGA TERMINATES NVIDIA PARTNERSHIP                 META    
2  [CONSOLE] STEAM DECK - 64GB/$359 256GB/$476 51...               CONSOLE   
3  [META] NVIDIA GEFORCE RTX 4090 24GB GDDR6X TO ...                 META    
4  [MONITOR] DELL 27” S2721DGF 1440P 165HZ IPS 1M...  EXPIRED :TABLE_FLIP:   

   Score   Created UTC                                                URL  \
0   3732  1.678456e+09  https://www.pcmag.com/news/electronics-retaile...   
1   2960  1.663356e+09        https://www.youtube.com/watch?v=cV9QES-FUAM   
2   2067  1.678986e+09           https://store.steampowered.com/steamdeck   
3   2023  1.663688e+09  https://www.nvidia.com/en-us/geforce/graphics-...   
4   2005  1.666010e+09  https://www.dell.com/en-us/shop/dell-27-gaming...   

          Author  Num_Comments Created Date             Website  
0 

Extract Sub Flairs in Title column

In [77]:
# Extract text between square brackets and create "flair" column
df['Title Flair'] = df['Title'].apply(lambda x: re.findall(r'\[(.*?)\]', x)[0] if re.findall(r'\[(.*?)\]', x) else None)

Clean up Flair Column

In [78]:
# Get all unique values in the "flair" column
unique_flairs = df['Flair'].unique()

print(unique_flairs)


['META ' 'CONSOLE' 'EXPIRED :TABLE_FLIP:' 'GPU' 'MISCELLANEOUS' 'MOD POST'
 'CPU' 'EXPIRED' 'HEADPHONES' 'SSD - M.2' 'FURNITURE' 'CONTROLLER' 'OTHER'
 'MONITOR' 'BUNDLE' 'PRINTER' 'CABLES' 'CASE' 'SSD - SATA' 'MOUSE' 'HDD'
 'LAPTOP' 'RAM' 'SPEAKERS' 'PREBUILT' 'KEYBOARD' None 'PSU' 'NETWORKING'
 'HEADSET' 'VR' 'MOTHERBOARD' 'FLASH DRIVE' 'COOLER' 'OPTICAL DRIVE' 'FAN'
 'WEBCAM']


Change "Explired flairs" with flair extracted from title

In [79]:

# Update "flair" column if value is "EXPIRED :TABLE_FLIP:"
df.loc[df['Flair'] == 'EXPIRED :TABLE_FLIP:', 'Flair'] = df.loc[df['Flair'] == 'EXPIRED :TABLE_FLIP:', 'Title Flair']

# Update "flair" column if value is "EXPIRED :TABLE_FLIP:"
df.loc[df['Flair'] == 'EXPIRED', 'Flair'] = df.loc[df['Flair'] == 'EXPIRED', 'Title Flair']

# Change "flair" column values to uppercase
df['Flair'] = df['Flair'].str.upper()

# Get all unique values in the "flair" column
unique_flairs = df['Flair'].unique()

df['Flair'] = df['Flair'].str.upper()
print(unique_flairs)


['META ' 'CONSOLE' 'MONITOR' 'GPU' 'CONTROLLER' 'MISCELLANEOUS' 'SSD'
 'MOD POST' 'CPU' 'PREBUILT' 'GPUS' 'OS' 'HEADPHONES' 'SSD - M.2'
 'FURNITURE' 'NVME' 'OTHER' 'RX 6600 XT' 'NVME SSD' 'BUNDLE' 'POWER STRIP'
 'SPEAKERS'
 'LG 27” ULTRAGEAR UHD NANO IPS 1MS 144HZ G-SYNC COMPATIBLE GAMING MONITOR WITH HDR (DISPLAYPORT, HDMI, USB) BLACK 27GN950-B'
 'PRINTER' 'CABLES' 'CABLE' 'CASE' 'GAMING WHEEL' 'HDD' 'RAM' 'SSD - SATA'
 'MOUSE' 'PSU' 'LAPTOP' 'GAMING LAPTOP' 'MONITOR ARM'
 'LG 27" ULTRAGEAR UHD NANO IPS 1MS 144HZ G-SYNC COMPATIBLE GAMING MONITOR WITH HDR (DISPLAYPORT, HDMI, USB) BLACK 27GN950-B'
 'TV' 'COMBO' 'CHAIR' 'MICROPHONE' 'KEYBOARD' ' PREBUILT ' None 'M.2 SSD'
 'CPU COOLER' ' MOTHERBOARD ' 'CPU+MOBO' 'COOLER' ' MONITOR ' 'GIVEAWAY'
 'NETWORKING' 'SSD-M.2' 'HEADSET' 'SATA SSD' 'VR' 'MOTHERBOARD'
 'FLASH DRIVE' 'MOUSE PAD' 'MICRO SD' 'EVGA' 'GRAPHICS CARD' 'WEBCAM'
 'AIO COOLER' 'MISC' 'MONITOR/TV' 'ASUS TUF 6800XT' 'MONITOR MOUNT'
 'EXTERNAL SSD' 'VR HEADSET' 'DUSTER' 'OPTICAL 

clean up the new flairs

In [82]:
# group items'
df['Flair'] = df['Flair'].replace(['LG 27" UltraGear UHD Nano IPS 1ms 144Hz G-SYNC Compatible Gaming Monitor with HDR (DisplayPort, HDMI, USB) Black 27GN950-B',
                                   'LG 27" ULTRAGEAR UHD NANO IPS 1MS 144HZ G-SYNC COMPATIBLE GAMING MONITOR WITH HDR (DISPLAYPORT, HDMI, USB) BLACK 27GN950-B',
                                   'LG 27” ULTRAGEAR UHD NANO IPS 1MS 144HZ G-SYNC COMPATIBLE GAMING MONITOR WITH HDR (DISPLAYPORT, HDMI, USB) BLACK 27GN950-B',
                                   ' MONITOR ', 'MONITOR/TV', 'TV' ],
                                  'MONITOR')

df['Flair'] = df['Flair'].replace(['GPUS', 'RX 6600 XT', 'EVGA', 'GRAPHICS CARD', 'ASUS TUF 6800XT',  ],
                                  'GPU')

df['Flair'] = df['Flair'].replace(['CASE', 'RAM', 'PSU', 'KEYBOARD', 'CPU COOLER', 'COOLER', 'AIO COOLER', 'OPTICAL DRIVE', 'FAN', 'OS', 'MOUSE', ' MOTHERBOARD ', 'MOTHERBOARD'  ],
                                  'PC PARTS')

df['Flair'] = df['Flair'].replace(['SSD', 'SSD - M.2', 'NVME', 'NVME SSD', 'HDD', 'SSD - SATA', 'M.2 SSD', 'SATA SSD', 'FLASH DRIVE', 'SSD-M.2', 'EXTERNAL SSD'  ],
                                  'STORAGE')

df['Flair'] = df['Flair'].replace(['CONTROLLER', 'FURNITURE', 'OTHER', 'POWER STRIP', 'PRINTER', 'CABLES', 'CABLE', 'GAMING WHEEL', 'MONITOR ARM', 'CHAIR', 'VR', 'MOUSE PAD', 'MICRO SD', 'WEBCAM', 'MISC', 'MONITOR MOUNT', 'VR HEADSET', 'DUSTER', 'ROUTER', 'NETWORKING'  ],
                                  'MISCELLANEOUS')

df['Flair'] = df['Flair'].replace(['HEADPHONES', 'SPEAKERS', 'MICROPHONE', 'HEADSET', 'ASUS TUF 6800XT',  ],
                                  'AUDIO')

df['Flair'] = df['Flair'].replace(['BUNDLE', 'COMBO', ' PREBUILT ', 'CPU+MOBO',  ],
                                  'BUNDLE')

df['Flair'] = df['Flair'].replace(['PREBUILT', 'LAPTOP', 'GAMING LAPTOP'],
                                  'PC or LAPTOP')

# Drop rows with specific flair values
flairs_to_drop = ['META', 'MOD POST', None, 'GIVEAWAY', 'PRODUCT TYPE', 'META ']
df = df[~df['Flair'].isin(flairs_to_drop)]

# Get all unique values in the "flair" column
unique_flairs = df['Flair'].unique()

print(unique_flairs)

['CONSOLE' 'MONITOR' 'GPU' 'MISCELLANEOUS' 'STORAGE' 'CPU' 'PC or LAPTOP'
 'PC PARTS' 'AUDIO' 'BUNDLE']


In [83]:
# Export DataFrame to CSV file
csv_path = r'C:\Users\minty\OneDrive\Desktop\Python Pandas\data.csv'
df.to_csv(csv_path, index=False)